# Zc-Curve Samples

This notebook demonstrates how to access and use the zc-curves end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-ir-swap-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [3]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
# IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
zccurve_def_endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/zc-curve-definitions")
zccurve_endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/zc-curves")
print("session opened and end-point set")

session opened and end-point set
2021-01-25 22:24:26,595 - Session session.platform - Thread 28096 | MainThread
[Error 400 - invalid_grant] empty error description
2021-01-25 22:24:26,595 - Session session.platform - Thread 28096 | MainThread
[Error 400 - invalid_grant] empty error description
2021-01-25 22:24:26,595 - Session session.platform - Thread 28096 | MainThread
[Error 400 - invalid_grant] empty error description


#### retrieving the zc-curve id

The zc-curve is retrieved using the curve id provided by the zc-curve definition

In [5]:
import pandas as pd

request_body={
  "universe": [
    {
        "currency":"EUR"
    }
  ]
}

response = zccurve_def_endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

df = pd.DataFrame(data=response.data.raw["data"][0]["curveDefinitions"])
df

,currency,mainConstituentAssetClass,riskType,indexName,source,name,firstHistoricalAvailabilityDate,id,availableTenors,availableDiscountingTenors
0,EUR,Swap,InterestRate,EURIBOR,Refinitiv,EUR EURIBOR Swap ZC Curve,2008-07-14,9d619112-9ab3-45c9-b83c-eb04cbec382e,"[6M, 1Y, OIS, 3M, 1M]","[6M, 1Y, OIS, 3M, 1M]"


##### bootstrapping the curve

using the id of the first curve definition retrieved above, we can compute the zc-cure for the 6M basis using OIS discounting. we then display the curve in a data frame including:
- the tenor of the zero coupon rate
- the start date of the zero coupon period
- the end date of the zero coupon period
- the zero coupon rate
- the discount factor equivalent to the zero coupon rate
- the instrument if any used to bootstrap the tenor

In [12]:
request_body = {
    "universe": [
        {
            "curveDefinition": {
                "id": df['id'][0],
                "discountingTenor": "OIS",
                "indexTenors":["6M"]
            },
            "curveParameters":{
                "interpolationMode": "CubicSpline",
                "priceSide": "Mid", 
                "interestCalculationMethod": "Dcb_Actual_Actual", 
                "extrapolationMode": "Linear",
            }
        }],

     "outputs": ['DetailedCurvePoint']

}

response = zccurve_endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)


curve = pd.DataFrame(data=response.data.raw["data"][0]["curves"]["6M"]["curvePoints"])
curve = curve[['tenor','startDate','endDate','ratePercent','discountFactor','instruments']]
curve

,tenor,startDate,endDate,ratePercent,discountFactor,instruments
0,0D,2021-01-25,2021-01-25,-0.557610,1.000000,NaN
1,ON,2021-01-25,2021-01-26,-0.557610,1.000015,"[{'instrumentCode': 'EUROND=', 'value': -0.55}]"
2,2D,2021-01-25,2021-01-27,-0.557610,1.000031,NaN
3,6M,2021-01-25,2021-07-27,-0.528845,1.002662,[{'instrumentCode': 'EUR0X6F=SMKR'}]
4,7M,2021-01-25,2021-08-27,-0.529694,1.003119,[{'instrumentCode': 'EUR1X7F=SMKR'}]
5,8M,2021-01-25,2021-09-27,-0.530090,1.003574,[{'instrumentCode': 'EUR2X8F=SMKR'}]
6,9M,2021-01-25,2021-10-27,-0.530085,1.004012,[{'instrumentCode': 'EUR3X9F=SMKR'}]
7,10M,2021-01-25,2021-11-29,-0.530285,1.004497,[{'instrumentCode': 'EUR4X10F=SMKR'}]
8,11M,2021-01-25,2021-12-27,-0.529935,1.004903,[{'instrumentCode': 'EUR5X11F=SMKR'}]
9,1Y,2021-01-25,2022-01-27,-0.529215,1.005349,[{'instrumentCode': 'EUR6X12F=SMKR'}]
